<a href="https://colab.research.google.com/github/ManuelPer/New_repo/blob/main/SAS_PV/SAS_PV_esercitazioni_EV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Installa le librerie necessarie
%%capture
!pip install ipywidgets pvlib
import ipywidgets as widgets
import pandas as pd
import numpy as np
import gdown
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from pvlib import location, pvsystem
from pvlib.iotools import get_pvgis_hourly
from pvlib.modelchain import ModelChain
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
from pvlib.iotools import get_pvgis_tmy
from IPython.display import display, clear_output, Javascript
from google.colab import output

# **Dimensionamento di un sistema di ricarica monofamiliare**

Con questo esercizio si può valutare la potenza necessaria per ricaricare un veicolo elettrico in casa, considerando:

- Efficienza dell’auto (kWh/100km)  
- Chilometri giornalieri  
- Frequenza e durata della ricarica  
- Efficienza del caricatore  

Viene calcolata l’**energia necessaria** e la **potenza minima richiesta**, posteriormente confrontata con le potenze standard disponibili.

**Formule principali:**

`Energia = (km/giorno × eff.auto / 100) / (eff. caricatore / 100) × giorni tra ricariche`  
`Potenza = Energia / ore disponibili`

L’obiettivo è capire come abitudini e parametri tecnici influenzano la scelta del sistema di ricarica.




In [ ]:
# @title Codice primo esercizio

layout=widgets.Layout(width='500px')
style_descr = {'description_width': 'initial'}
# Slider per l'utente singolo
efficienza_auto_slider = widgets.FloatSlider(value=20, min=10, max=30, step=5, description='Efficienza auto (kWh/100km)',style=style_descr,layout=layout)
km_giornalieri_slider = widgets.FloatSlider(value=30, min=20, max=100, step=10, description='km/giorno',style=style_descr,layout=layout)
tempo_ricarica_slider = widgets.FloatSlider(value=6, min=3, max=12, step=1, description='Tempo ricarica minimo disponibile(h)',style=style_descr,layout=layout)
efficienza_caricatore_slider = widgets.FloatSlider(value=90, min=80, max=100, step=5, description='Efficienza caricatore (%)',style=style_descr,layout=layout)
giorni_carica_slider = widgets.FloatSlider(value=1, min=1, max=4, step=1, description='Giorni fra le ricariche',style=style_descr,layout=layout)
# potenza_rete_slider = widgets.FloatSlider(value=3.3, min=2.2, max=7.4, step=0.1, description='Potenza rete (kW)')
potenze_disponibili = [3.7, 7.4, 11, 22]
# Funzione per calcolare e visualizzare i risultati utente singolo
def calcola_ricarica_utente(consumo_auto, km_giornalieri, tempo_ricarica, efficienza_caricatore, giorni_fra_le_ricariche):
    energia_necessaria = km_giornalieri * consumo_auto /100 / (efficienza_caricatore/100) * giorni_fra_le_ricariche
    potenza_necessaria = energia_necessaria / tempo_ricarica

    # Potenze standard disponibili

    # Suggerisci la potenza più vicina che sia >= potenza necessaria
    potenza_suggerita = next((p for p in potenze_disponibili if p >= potenza_necessaria), "oltre 22 kW")

    print(f"Energia necessaria: {energia_necessaria:.2f} kWh")
    print(f"Potenza di ricarica minima necessaria: {potenza_necessaria:.2f} kW")
    print(f"Potenza suggerita per la stazione di ricarica: {potenza_suggerita} kW")


# Interazione con le slider per utente singolo
widgets.interact(calcola_ricarica_utente,
                  consumo_auto=efficienza_auto_slider,
                  km_giornalieri=km_giornalieri_slider,
                  tempo_ricarica=tempo_ricarica_slider,
                  efficienza_caricatore=efficienza_caricatore_slider,
                  giorni_fra_le_ricariche=giorni_carica_slider
                  )

# **Valutazione dei Sistemi di Gestione Intelligente del Carico**

In questo esercizio avrai la possibilità di simulare il comportamento energetico di un'utenza domestica con un impianto fotovoltaico e un veicolo elettrico, modificando variabili come strategia di ricarica, consumo domestico, orari di parcheggio, potenza della colonnina, dimensione dell’impianto PV e limite di potenza ( allacciamento).

Sono disponibili cinque strategie di ricarica:


*   **Non controllato**
*   **solar-green 1**: caricare solo quando c'è un surplus PV
*   **solar-green 2**: priorizzare il surplus del PV per la caricare, ma guarantire la soddisfazione dell'utente
*   **tariffa**: priorizzare la ricarica durante le fasce tariffarie più convenienti, ma guarantire la soddisfazione dell'utente
*   **Combinato** (solar 2 +tariffa)

L'obiettivo è analizzare come scelte tecniche e comportamentali influenzano costi, autoconsumo e qualità della ricarica.


In [ ]:
# @title Carica profili secondo esercizio
!gdown 1-Fqx9RxTZ_kSKSVZW6E8mfT5BRscciKl
profilo_carico_domestici = pd.read_parquet('profili_SAS.parquet')

profilo_carico_domestici.columns = ['basso', 'average', 'alto', 'molto alto']


In [ ]:
# @title Codice seconda esercizio
output.enable_custom_widget_manager()

# Cell 2: Widget definitions
layout = widgets.Layout(width='500px')
style_descr = {'description_width': 'initial'}

# Create widgets
user_type = widgets.Dropdown(
    options=['basso', 'average', 'alto', 'molto alto'],
    value='average',
    description='Tipo utente:'
)

pv_install = widgets.Checkbox(
    value=True,
    description='Installazione PV'
)

pv_power = widgets.FloatSlider(
    min=3.0,
    max=15.0,
    step=0.5,
    value=5.0,
    description='Potenza PV (kWp):',
    style=style_descr, layout=layout
)

lat = widgets.FloatText(
    value=45.933,
    description='Latitudine:'
)

lon = widgets.FloatText(
    value=9.0,
    description='Longitudine:'
)

algorithm = widgets.Dropdown(
    options=['not_control', 'solar-green1', 'solar-green2', 'tariffa', 'combinato'],
    value='not_control',
    description='Algoritmo:'
)

max_power = widgets.FloatSlider(
    min=16.0,
    max=32.0,
    step=2.0,
    value=16.0,
    description='Max potenza casa (kW):',
    style=style_descr, layout=layout
)

vehicle_consumption = widgets.FloatSlider(
    min=2.0,
    max=24.0,
    step=2.0,
    value=6.0,
    description='Consumo veicolo (kWh/giorno):',
    style=style_descr, layout=layout
)

charger_power = widgets.Dropdown(
    options=[3.7, 7.4, 11],
    value=3.7,
    description='Potenza colonnina (kW):',
    style=style_descr,layout=layout
)

start_day_plot = widgets.FloatSlider(
    min=1.0,
    max=365.0,
    step=1.0,
    value=17.0,
    description='Primo giorno graficato',
    style=style_descr, layout=layout
)

number_days_plot = widgets.FloatSlider(
    min=1.0,
    max=7.0,
    step=1.0,
    value=2.0,
    description='Numero di giorni graficati',
    style=style_descr, layout=layout
)

min_energy_need_for_charging = widgets.FloatSlider(
    min=0.0,
    max=15.0,
    step=5.0,
    value=0.0,
    description='Fabbisogno d energia minimo per iniziare la carica',
    style=style_descr, layout=layout
)

# Time grid creation function
def create_time_grid(day_name):
    defaults = {
        "Feriali": list(range(0, 8)) + list(range(19, 24)),
        "Sabato": list(range(0, 8)) + list(range(19, 24)),
        "Domenica": list(range(0, 16)) + list(range(17, 24))
    }
    preselected = defaults.get(day_name, [])

    hours = [widgets.ToggleButton(
        value=i in preselected,
        layout={'width': '30px', 'height': '30px'},
        style={'button_color': 'lightgreen' if i in preselected else 'lightgray'}
    ) for i in range(24)]

    def on_toggle(change):
        if change['new']:
            change['owner'].style.button_color = 'lightgreen'
        else:
            change['owner'].style.button_color = 'lightgray'

    for hour in hours:
        hour.observe(on_toggle, 'value')

    return widgets.VBox([
        widgets.HTML(f"<b>{day_name}</b>"),
        widgets.HBox([
            widgets.VBox([
                widgets.Label(str(i)),
                hours[i]
            ]) for i in range(24)
        ])
    ])

# Create time grids
weekday_grid = create_time_grid("Feriali")
saturday_grid = create_time_grid("Sabato")
sunday_grid = create_time_grid("Domenica")

# Assemble parameter section
time_selector = widgets.VBox([
    weekday_grid,
    saturday_grid,
    sunday_grid
])

parametri = widgets.VBox([
    widgets.HTML("<b>Configura il sistema:</b>"),
    widgets.HBox([user_type,vehicle_consumption,pv_install]),
    widgets.HBox([pv_power, lat, lon]),
    widgets.HBox([algorithm, max_power, charger_power]),
    widgets.HBox([min_energy_need_for_charging]),
    widgets.VBox([
        widgets.HTML("<b>Seleziona orari presenza EV:</b>"),
        time_selector
    ]),
    widgets.VBox([
        widgets.HTML("<b>Configura i grafici:</b>"),
        widgets.HBox([start_day_plot, number_days_plot])
    ])
])

display(parametri)

# Cell 3: Interactive calculation and plotting
out = widgets.Output()
display(out)

def get_selected_hours(grid):
    selected = []
    start = None
    for i, btn in enumerate(grid.children[1].children):
        if btn.children[1].value:
            if start is None:
                start = i
        else:
            if start is not None:
                selected.append([start, i])
                start = None
    if start is not None:
        selected.append([start, 24])
    return selected

def update_calculation():
    with out:
        clear_output(wait=True)

        # Get all current widget values
        weekday_intervals = get_selected_hours(weekday_grid)
        saturday_intervals = get_selected_hours(saturday_grid)
        sunday_intervals = get_selected_hours(sunday_grid)

        # Constants and data initialization
        times = pd.date_range('2023-01-01', '2023-12-31 23:45', freq='15min')


        # 1. Household load
        # user_loads = {'basso':0.5, 'average':1.0, 'alto':1.5, 'molto alto':2.0}
        # base_load = pd.Series(user_loads[user_type.value], index=times)
        a = np.array(profilo_carico_domestici[user_type.value])
        base_load = pd.Series(a, index=times)

        # 2. PV Generation
        if pv_install.value:
            system = pvsystem.PVSystem(
                surface_tilt=13,
                surface_azimuth=180,
                module_parameters={'pdc0': pv_power.value*1000, 'gamma_pdc': -0.004},
                inverter_parameters={'pdc0': pv_power.value*1000},
                temperature_model_parameters=TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']
            )
            loc = location.Location(lat.value, lon.value)
            #weather = loc.get_clearsky(times)

            tmy_data, months_selected, inputs, metadata = get_pvgis_tmy(
                latitude=lat.value,
                longitude=lon.value,
                usehorizon=True

            )

            weather = pd.DataFrame({
                'dni': tmy_data['dni'],
                'dhi': tmy_data['dhi'],
                'ghi': tmy_data['ghi'],
                'temp_air': tmy_data['temp_air'],
                'wind_speed': tmy_data['wind_speed'],
            })

            weather.index = pd.date_range('2023-01-01', '2023-12-31 23:45', freq='h')
            weather = weather.reindex(times).interpolate().ffill().bfill()

            mc = ModelChain(system, loc,
                    aoi_model='physical',  # spectral model 'no loss' otherwise error
                    temperature_model='sapm', losses_model='pvwatts')

            mc.run_model(weather)
            pv_gen = mc.results.ac.fillna(0) / 1000
            pv_gen = pv_gen.tz_localize('UTC').tz_convert('Europe/Rome')
        else:
            pv_gen = pd.Series(0, index=times)
            # pv_gen = np.zeros(len(times))
            # pv_gen = pd.Series(pv_gen)

        # 3. Energy pricing
        price = np.select(
            [times.hour.isin(range(23,24)) | times.hour.isin(range(0,6)),
             times.hour.isin(range(12,14))],
            [0.17, 0.24], default=0.32)
        price_series = pd.Series(price, index=times)

        # Initialize an array to hold tariff categories (0=low, 1=medium, 2=high)
        tariff_category = np.zeros(len(times), dtype=int)

        # Group by day and assign categories dynamically
        for day, day_prices in price_series.groupby(price_series.index.date):
            sorted_prices = day_prices.sort_values()
            n = len(day_prices)

            # Define cutoffs: 33% and 66%
            cut1 = sorted_prices.iloc[int(n * 1/3)]
            cut2 = sorted_prices.iloc[int(n * 2/3)]

            # Assign category
            for idx in day_prices.index:
                p = price_series[idx]
                if p < cut1:
                    tariff_category[price_series.index.get_loc(idx)] = 0  # cheap
                elif p < cut2:
                    tariff_category[price_series.index.get_loc(idx)] = 1  # medium
                else:
                    tariff_category[price_series.index.get_loc(idx)] = 2  # expensive

        # EV availability calculation
        def ev_availability(times, weekday_hours, saturday_hours, sunday_hours):
            at_home = np.zeros(len(times), dtype=bool)
            def in_intervals(hour, intervals):
                for start, end in intervals:
                    if start <= hour < end:
                        return True
                return False

            for i, t in enumerate(times):
                if t.weekday() < 5:
                    intervals = weekday_hours
                elif t.weekday() == 5:
                    intervals = saturday_hours
                else:
                    intervals = sunday_hours
                at_home[i] = in_intervals(t.hour, intervals)
            return at_home

        ev_home = ev_availability(times, weekday_intervals, saturday_intervals, sunday_intervals)

        # 4. EV charging simulation
        np.random.seed(42)
        unique_days = pd.to_datetime(np.unique(times.date))
        base_daily = vehicle_consumption.value  # kWh
        daily_needs = []
        for day in unique_days:
            weekday = day.weekday()  # 0 = Monday, ..., 6 = Sunday

            if weekday < 5:  # Monday to Friday
                variation = np.random.uniform(0.7, 1.3)
            elif weekday == 5:  # Saturday
                variation = np.random.uniform(1.1, 1.5)
            else:  # Sunday
                variation = np.random.uniform(0.3, 0.7)

            daily_needs.append(base_daily * variation)
        # daily_needs = np.array(daily_needs[:365])
        daily_needs = np.array(daily_needs)

        vehicle_load = np.zeros(len(times))
        for day in range(365):
            day_start = day * 96
            day_slice = slice(day_start, day_start + 96)
            away_mask = ~ev_home[day_slice]
            if away_mask.any():
                daily_energy = daily_needs[day] / away_mask.sum()
                vehicle_load[day_slice] += np.where(away_mask, daily_energy, 0)

        # Simulation variables
        autoconsumo_ev = 0
        costo_energia_ev = 0
        costo_energia_casa = 0
        autoconsumo_casa = 0
        pv_export = 0
        energy_need = 0
        charge_power = np.zeros(len(times))
        pv_used_ev = np.zeros(len(times))
        grid_used_ev = np.zeros(len(times))
        grid_used_house = np.zeros(len(times))
        energy_need_vector = np.zeros(len(times))
        pv_export_vector = np.zeros(len(times))

        charging_active = False

        # Main simulation loop
        for i in range(len(times)):
            house_grid_usage = max(base_load.iloc[i] - pv_gen.iloc[i], 0)
            grid_used_house[i] = house_grid_usage
            costo_energia_casa += house_grid_usage * 0.25 * price_series.iloc[i]

            if not ev_home[i]:
                energy_need += vehicle_load[i]
                charging_active = False

            if ev_home[i] and energy_need > 0:
                pv_surplus = max(pv_gen.iloc[i] - base_load.iloc[i], 0)
                grid_available = max_power.value - house_grid_usage

                # Control algorithm logic
                if algorithm.value == 'not_control':
                    charge = True
                    max_charge = min(
                        charger_power.value,
                        grid_available + pv_surplus,
                        energy_need / 0.25
                        )
                elif algorithm.value == 'solar-green1':
                    charge = (pv_surplus > 0)
                    max_charge = min(
                        charger_power.value,
                        pv_surplus,
                        energy_need / 0.25
                    )
                elif algorithm.value == 'solar-green2':
                  # Trova il prossimo index in cui l'EV non è più connesso (False)
                  remaining_time = 0
                  j=i
                  while j < len(ev_home) and ev_home[j]:
                      remaining_time += 1
                      j += 1
                  # print(remaining_time)

                  # Calcolare il tempo necessario per completare la carica
                  required_time_for_charge = energy_need / (charger_power.value)  # Tempo necessario per caricare con la potenza massima
                  # Se il tempo rimanente è sufficiente per completare la carica, carica con energia solare
                  if remaining_time -1 >= required_time_for_charge *4 :
                      charge = (pv_surplus > 0)  # Se c'è surplus di PV, carica con PV
                      max_charge = min(
                        charger_power.value,
                        pv_surplus,
                        energy_need / 0.25
                    )
                  else:
                      # Se non c'è abbastanza tempo, carica dalla rete
                      charge = True  # Usa energia dalla rete
                      max_charge = min(
                        charger_power.value,
                        grid_available + pv_surplus,
                        energy_need / 0.25
                    )


                elif algorithm.value == 'tariffa':
                    remaining_time = 0
                    j = i
                    while j < len(ev_home) and ev_home[j]:
                        remaining_time += 1
                        j += 1
                    required_time_for_charge = energy_need / (charger_power.value)  # Tempo necessario per caricare con la potenza massima
                    # Se il tempo rimanente è sufficiente per completare la carica, carica con energia solare
                    max_charge = min(
                        charger_power.value,
                        grid_available + pv_surplus,
                        energy_need / 0.25
                    )
                    if remaining_time -1 >= required_time_for_charge *4 :
                        # Allow charge only in low or medium tariff category
                        charge = tariff_category[i] < 1  # Change to == 0 for more aggressive
                    else:
                        # Se non c'è abbastanza tempo, carica dalla rete
                        charge = True  # Usa energia dalla rete

                elif algorithm.value == 'combinato':
                    remaining_time = 0
                    j = i
                    while j < len(ev_home) and ev_home[j]:
                        remaining_time += 1
                        j += 1
                    # Calcolare il tempo necessario per completare la carica
                    required_time_for_charge = energy_need / (charger_power.value)  # Tempo necessario per caricare con la potenza massima
                    # Se il tempo rimanente è sufficiente per completare la carica, carica con energia solare
                    if remaining_time -1 >= required_time_for_charge *4 :
                        # Allow charge only in low or medium tariff category
                        charge = (tariff_category[i] < 1) or (pv_surplus > 0)  # Change to == 0 for more aggressive
                        if (tariff_category[i] < 1):
                            max_charge = min(
                            charger_power.value,
                            grid_available + pv_surplus,
                            energy_need / 0.25
                            )
                        else:
                            max_charge = min(
                            charger_power.value,
                            pv_surplus,
                            energy_need / 0.25
                            )
                    else:
                        # Se non c'è abbastanza tempo, carica dalla rete
                        max_charge = min(
                        charger_power.value,
                        grid_available + pv_surplus,
                        energy_need / 0.25
                        )
                        charge = True  # Usa energia dalla rete
                else:
                    charge = False

                if not charging_active:
                    if energy_need < min_energy_need_for_charging.value:
                        charge = False
                    else:
                        charging_active = True


                if charge:
                    # max_charge = min(
                    #     charger_power.value,
                    #     grid_available + pv_surplus,
                    #     energy_need / 0.25
                    # )
                    pv_used = min(max_charge, pv_surplus)
                    grid_used = max_charge - pv_used

                    charge_power[i] = max_charge
                    pv_used_ev[i] = pv_used
                    grid_used_ev[i] = grid_used

                    autoconsumo_ev += pv_used * 0.25
                    costo_energia_ev += grid_used * 0.25 * price[i]
                    energy_need -= max_charge * 0.25

            pv_for_house = min(pv_gen.iloc[i], base_load.iloc[i])
            autoconsumo_casa += pv_for_house * 0.25
            pv_export += max(pv_gen.iloc[i] - base_load.iloc[i] - pv_used_ev[i], 0) * 0.25
            pv_export_vector[i] = max(pv_gen.iloc[i] - base_load.iloc[i] - pv_used_ev[i], 0) * 0.25
            energy_need_vector[i] = energy_need

        # Results calculation
        costo_totale = costo_energia_ev + costo_energia_casa
        total_pv_production = np.sum(pv_gen) * 0.25
        total_ev_energy_required = np.sum(vehicle_load)
        total_ev_energy_charged = np.sum(charge_power) * 0.25

        # Display results
        print(f"\nAutoconsumo casa: {autoconsumo_casa:.1f} kWh")
        print(f"Autoconsumo EV: {autoconsumo_ev:.1f} kWh")
        print(f"PV esportato: {pv_export:.1f} kWh")
        print(f"Produzione totale PV: {total_pv_production:.1f} kWh")

        print(f"\nCosto energia casa: {costo_energia_casa:.1f} CHF")
        print(f"Costo energia EV: {costo_energia_ev:.1f} CHF")
        print(f"Costo totale energia: {costo_totale:.1f} CHF")
        consumo_totale_casa = sum(base_load)*0.25
        print(f"\nEnergia richiesta casa: {consumo_totale_casa:.1f} kWh")
        print(f"\nEnergia richiesta EV: {total_ev_energy_required:.1f} kWh")
        print(f"Energia caricata EV: {total_ev_energy_charged:.1f} kWh")
        if total_ev_energy_required - total_ev_energy_charged > 0.1 + min_energy_need_for_charging.value:
            print(f"\nEnergia non caricata: {total_ev_energy_required - total_ev_energy_charged:.1f} kWh")
            print("ATTENZIONE: CIÒ SIGNIFICA CHE L'UTENTE NON POTRÀ EFFETTUARE TUTTI I VIAGGI DESIDERATI O DOVRÀ RICARICARE FUORI CASA.")

        # Plotting
        def plot_detailed(n,m):
            fig, ax = plt.subplots(7, 1, figsize=(15, 20))

            # Create the index for the desired range of days
            start_idx = int(max((n-1) * 96,364))  # Assuming 96 time steps per day
            end_idx = int(max(start_idx + (m * 96),364))

            x = times[start_idx:end_idx]
            pv_house_vector = pv_gen-pv_export_vector/0.25-pv_used_ev

            ax[0].plot(x,base_load[start_idx:end_idx], label='Carico casa')
            ax[0].plot(x,pv_gen[start_idx:end_idx], label='Produzione PV')
            ax[0].legend()
            ax[0].set_title('Carico vs Generazione')
            ax[0].set_ylabel('kW')

            ax[1].plot(x,grid_used_house[start_idx:end_idx], '--',label='Potenza rete casa')
            ax[1].plot(x,pv_house_vector[start_idx:end_idx] ,'--', label='Potenza PV casa')
            ax[1].plot(x,base_load[start_idx:end_idx], label='Potenza carico totale casa')
            ax[1].legend()
            ax[1].set_ylabel('kW')
            ax[1].set_title('Consumo Domestico')

            ax[2].plot(x,price[start_idx:end_idx], label='Prezzo energia')
            ax[2].legend()
            ax[2].set_title('Prezzo Energia')
            ax[2].set_ylabel('CHF/kWh')

            ax[3].step(x, ev_home[start_idx:end_idx], where='post')
            ax[3].set_yticks([0, 1])
            ax[3].set_yticklabels(['Fuori', 'Casa'])
            ax[3].set_title('Disponibilità EV')

            ax[4].plot(x,charge_power[start_idx:end_idx], label='Potenza totale di carica EV')
            ax[4].plot(x,pv_used_ev[start_idx:end_idx], '--',label='Quota PV')
            ax[4].plot(x,grid_used_ev[start_idx:end_idx], '--',label='Quota rete')
            ax2 = ax[4].twinx()  # Create a twin y-axis on the right side
            ax2.plot(x, energy_need_vector[start_idx:end_idx], label='Fabbisogno residuo', linestyle='--', color='red')
            ax2.set_ylabel('kWh', color='red')  # Set the label for the right y-axis
            ax2.tick_params(axis='y', labelcolor='red')  # Set color for the right y-axis ticks
            ax[4].legend()
            ax[4].set_title('Dettaglio Carica EV')
            ax[4].set_ylabel('kW')

            ax[5].plot(x, pv_gen[start_idx:end_idx], label='Produzione PV', color='orange')
            ax[5].plot(x, pv_used_ev[start_idx:end_idx] + pv_house_vector[start_idx:end_idx], label='Autoconsumo EV + Domestic', linestyle='--', color='blue')
            ax[5].legend()
            ax[5].set_title('Impianto PV')
            ax[5].set_ylabel('kW')


            ax[6].plot(x, np.ones(len(x))*max_power.value, label='Potenza massima ammissibile', color='red')
            ax[6].plot(x, grid_used_house[start_idx:end_idx] + grid_used_ev[start_idx:end_idx], label='Utilizzo totale della rete')
            ax[6].legend()
            ax[6].set_title('Utilizzo della capacità di rete')
            ax[6].set_ylabel('kW')

            plt.tight_layout()
            plt.subplots_adjust(hspace=0.4)
            for a in ax:
              plt.setp(a.get_xticklabels(), rotation=20)
            plt.show()

        plot_detailed(n=start_day_plot.value,m=number_days_plot.value)

# Cell 4: Set up observers
widgets_to_observe = [
    user_type, vehicle_consumption, pv_install, pv_power, lat, lon,
    algorithm, max_power, charger_power, min_energy_need_for_charging, start_day_plot, number_days_plot
]

# Collect all toggle buttons
toggle_buttons = []
for grid in [weekday_grid, saturday_grid, sunday_grid]:
    for hour_widget in grid.children[1].children:
        toggle_buttons.append(hour_widget.children[1])

# Attach observers to all relevant widgets
for widget in widgets_to_observe + toggle_buttons:
    widget.observe(lambda _: update_calculation(), names='value')

# Initial calculation
update_calculation()

display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

# **Dimensionamento di un sistema di ricarica per un condominio**

In [ ]:
# @title Codice dimensionamento
# Slider per il condominio
posti_auto_slider = widgets.IntSlider(value=10, min=5, max=40, step=5, description='Posti auto')
perc_auto_elettriche_slider = widgets.FloatSlider(value=0.2, min=0, max=1, step=0.1, description='% auto elettriche')
fattore_contemporaneita_slider = widgets.FloatSlider(value=0.3, min=0, max=1, step=0.1, description='Fattore contempor.')
potenza_punto_ricarica_slider = widgets.SelectionSlider(options=potenze_disponibili, value=3.7,description='Potenza ricarica (kW)', continuous_update=False, style={'description_width': 'initial'},layout=layout)

# potenza_condominio_slider = widgets.FloatSlider(value=50, min=20, max=200, step=10, description='Potenza condominio (kW)')

# Funzione per calcolare e visualizzare i risultati condominio
def calcola_ricarica_condominio(posti_auto, perc_auto_elettriche, potenza_per_auto, fattore_contemporaneita):
    numero_auto_elettriche = posti_auto * perc_auto_elettriche
    potenza_totale_necessaria = numero_auto_elettriche * potenza_per_auto * fattore_contemporaneita

    print(f"Potenza totale necessaria: {potenza_totale_necessaria:.2f} kW")

    # if potenza_totale_necessaria <= potenza_condominio:
    #     print("La potenza del condominio è sufficiente.")
    #     print("Si possono installare colonnine senza gestione del carico (ma è consigliata).")
    # else:
    #     print("Potenza insufficiente, è necessario un sistema di gestione del carico.")
    #     print("Valutare diverse opzioni: riduzione potenza colonnine, gestione dinamica, ecc.")

# Interazione con le slider per condominio
widgets.interact(calcola_ricarica_condominio,
                  posti_auto=posti_auto_slider,
                  perc_auto_elettriche=perc_auto_elettriche_slider,
                  fattore_contemporaneita=fattore_contemporaneita_slider,
                  potenza_per_auto=potenza_punto_ricarica_slider)